In [19]:
# Import Dependencies

import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import requests
import json
from census import Census, gkey
from pprint import pprint

# Import Census API keys
from config import census_key

In [146]:
# Read in original csv file as dataframe
accidents_csv = pd.read_csv("Resources/US_Accidents_Dec21_updated.csv")

# Display first 5 rows of dataframe
accidents_csv.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [149]:
# Create new column consisting of only the year
accidents_csv["Year"] = accidents_csv["Start_Time"].str[:4]

# Create new dataframe to bypass having to load original
accidents_df = accidents_csv

# Display first 5 rows
accidents_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,Night,Night,Night,Night,2016
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,Night,Night,Night,Night,2016
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,Night,Night,Night,Day,2016
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,Night,Night,Day,Day,2016
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2016


In [182]:
# Clean dataframe by limiting to selected columns
accidents_cleaned_df = accidents_df[['Start_Time', 'Start_Lat', 'Start_Lng',
       'Severity', 'Distance(mi)', 'County', 'State', 'Zipcode', 'Country',
       'Sunrise_Sunset', 'Year']]

# Display first 5 rows of cleaned df
accidents_cleaned_df.head()

,Start_Time,Start_Lat,Start_Lng,Severity,Distance(mi),County,State,Zipcode,Country,Sunrise_Sunset,Year
0,2016-02-08 00:37:08,40.108910,-83.092860,3,3.230,Franklin,OH,43017,US,Night,2016
1,2016-02-08 05:56:20,39.865420,-84.062800,2,0.747,Montgomery,OH,45424,US,Night,2016
2,2016-02-08 06:15:39,39.102660,-84.524680,2,0.055,Hamilton,OH,45203,US,Night,2016
3,2016-02-08 06:51:45,41.062130,-81.537840,2,0.123,Summit,OH,44311,US,Night,2016
4,2016-02-08 07:53:43,39.172393,-84.492792,3,0.500,Hamilton,OH,45217,US,Day,2016


In [153]:
# Create separate csv files for each year from 2016 to 2021

# List comprehension to create list of years included in datases cast as strings
years = [str(year) for year in range(2016,2022)]

# For loop to iterate through and export a separate csv file for each year
for yr in years:
    current_year = accidents_cleaned_df[accidents_cleaned_df["Year"] == yr]
    current_year.to_csv(f"Resources/accidents_{yr}.csv", index=False)

## Start from here!

In [2]:
# Designate selected year for dataframe
selected_year = "2019"

# Create Census object with Census API key from the selected year
c = Census(census_key, year=int(selected_year))

# Read in csv file for selected year as accidents_df
accidents_selected_year_df = pd.read_csv(f"Resources/accidents_{selected_year}.csv")

# Display first 5 rows of dataframe
accidents_selected_year_df.head()

,Start_Time,Start_Lat,Start_Lng,Severity,Distance(mi),County,State,Zipcode,Country,Sunrise_Sunset,Year
0,2019-10-02 12:50:00,40.736515,-74.084907,4,0.021,Hudson,NJ,07306,US,Day,2019
1,2019-11-01 16:18:00,29.955075,-90.071696,4,0.137,Orleans,LA,70112-2503,US,Day,2019
2,2019-06-04 16:00:00.000000000,29.773556,-95.351304,4,0.097,Harris,TX,77026-7228,US,Day,2019
3,2019-10-19 04:30:00.000000000,38.875540,-77.280396,4,1.483,Fairfax County,VA,22031,US,Night,2019
4,2019-12-17 09:12:52,33.634089,-112.106196,4,0.193,Maricopa,AZ,85023,US,Day,2019


In [3]:
county_count = accidents_selected_year_df.groupby(["State", "County"]).count()[["Start_Time"]]

county_count

Start_Time
State County                
AL    Autauga             10
      Baldwin             19
      Barbour              3
      Bibb                 6
      Blount              21
...                      ...
WY    Platte               5
      Sheridan             1
      Sublette             2
      Sweetwater          12
      Teton                2

[2087 rows x 1 columns]

In [4]:
# Select State, Country, and Start Lat/Lng columns as new dataframe
locations_df = accidents_selected_year_df[["State", "County", "Start_Lat", "Start_Lng"]]

# Merge dataframes to create new df consisting of the number of accidents in each county and the exact location of each
county_location_count = pd.merge(locations_df, county_count, on=["State", "County"])

In [5]:
#Rename columns
county_location_count.rename(columns={"Start_Time" : "Accident Count", 
                                        "Start_Lat" : "Lat", 
                                        "Start_Lng" : "Lng"}, inplace=True)

county_location_count.sort_values("State", inplace=True)

county_location_count.head()

,State,County,Lat,Lng,Accident Count
257813,AL,Dale,31.53060,-85.72338,6
255855,AL,Henry,31.35595,-85.17971,1
255861,AL,Fayette,33.69111,-87.82859,1
255862,AL,Geneva,31.15342,-85.64058,3
255863,AL,Geneva,31.18297,-86.14815,3


In [6]:
# Create new dataframe showing number of accidents and average coordinates by county

accidents_by_county = county_location_count.groupby(["State", "County"], as_index=False).mean()

accidents_by_county.head()

,State,County,Lat,Lng,Accident Count
0,AL,Autauga,32.549176,-86.501131,10
1,AL,Baldwin,30.687659,-87.845268,19
2,AL,Barbour,31.863403,-85.271253,3
3,AL,Bibb,33.001936,-87.117345,6
4,AL,Blount,33.943876,-86.610055,21


In [35]:
# Create dictionary for state abbreviations

states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [43]:
# Run Census Search to retrieve income data by county
census_data = c.acs5.get(("NAME", "B01003_001E", "B19013_001E","B19301_001E"), {'for': 'county:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Rename columns
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Median Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "NAME": "County"})

# Split "County" column into separate "County" and "State" columns
census_pd[["County", "State"]] = census_pd["County"].str.split(" County, ", expand=True)

# Convert State names to state abbreviations
census_pd.replace({"State": states}, inplace=True)

In [42]:
print(census_pd.head())
print(accidents_by_county.head())

    County  Population  Median Household Income  Per Capita Income state  \
0  Fayette     21565.0                  46650.0            23194.0    17   
1    Logan     29003.0                  57308.0            27546.0    17   
2   Saline     23994.0                  44090.0            25342.0    17   
3     Lake    701473.0                  89427.0            45766.0    17   
4   Massac     14219.0                  47481.0            23539.0    17   

  county     State  
0    051  Illinois  
1    107  Illinois  
2    165  Illinois  
3    097  Illinois  
4    127  Illinois  
  State   County        Lat        Lng  Accident Count
0    AL  Autauga  32.549176 -86.501131              10
1    AL  Baldwin  30.687659 -87.845268              19
2    AL  Barbour  31.863403 -85.271253               3
3    AL     Bibb  33.001936 -87.117345               6
4    AL   Blount  33.943876 -86.610055              21


In [50]:
# Merge census data with accident dataframe on State/County for counties listed in accidents_by_county dataframe
accidents_census_df = pd.merge(accidents_by_county, census_pd, on=["State","County"], how="left")

# Clean dataframe by removing unnecessary columns
accidents_census_df.drop(columns=["state","county"], inplace=True)

# Calculate accidents per capita and add to new column
accidents_census_df["Accidents (Per Capita)"] = accidents_census_df["Accident Count"] / accidents_census_df["Population"]

# Display first 5 rows of dataframe
accidents_census_df.head()

,State,County,Lat,Lng,Accident Count,Population,Median Household Income,Per Capita Income,Accidents (Per Capita)
0,AL,Autauga,32.549176,-86.501131,10,55380.0,58731.0,29819.0,0.000181
1,AL,Baldwin,30.687659,-87.845268,19,212830.0,58320.0,32626.0,0.000089
2,AL,Barbour,31.863403,-85.271253,3,25361.0,32525.0,18473.0,0.000118
3,AL,Bibb,33.001936,-87.117345,6,22493.0,47542.0,20778.0,0.000267
4,AL,Blount,33.943876,-86.610055,21,57681.0,49358.0,24747.0,0.000364


In [70]:
# Configure map plot
marker_size = accidents_census_df["Accidents (Per Capita)"] * 1000

# accident_county_map = accidents_census_df.hvplot.points(
#     "Lng",
#     "Lat",
#     geo = True,
#     tiles = "OSM",
#     frame_width = 800,
#     frame_height = 600,
#     scale = 1,
#     xlabel = "Longitude",
#     ylabel = "Latitude",
#     title = f"Accidents by County in {selected_year}",
#     alpha=0.5,
#     size = marker_size,
#     color = "purple",
#     hover_cols = ["County", "Accident Count", "Accidents (Per Capita)"]
#     )

accident_county_map = accidents_census_df.hvplot.heatmap(
    cmap="seismic",
    width = 800,
    height = 600

    )
    
# Display the map
accident_county_map

:HeatMap   [columns,index]   (value)